In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

cstmr = pd.read_csv("customer_data.csv") #Data from Company Database

# Visits and Survival

In [23]:
vst = pd.read_csv("all_vsts_202401.csv")
vst['ENTER_DATE'] = pd.to_datetime(vst['ENTER_DATE'])
vst['ENTER_DATE'] = vst['ENTER_DATE'].dt.date
vst = vst[["USER_ID", "ENTER_DATE"]].drop_duplicates()
vst['ENTER_DATE'] = pd.to_datetime(vst['ENTER_DATE'])

dimension = pd.DataFrame(vst["USER_ID"].unique(), 
                         columns=["USER_ID"]).merge(pd.DataFrame(week["Week_Number"].unique(), 
                                                                 columns=["Week_Number"]), how="cross")

first_visit_week = vst.merge(week, left_on="ENTER_DATE", right_on= "date", how="outer").groupby(["USER_ID"])["Week_Number"]\
                    .min().reset_index()

first_visit_week.columns = ["USER_ID", "first_visit_week"]

weekly = vst.merge(week, left_on="ENTER_DATE", right_on= "date", how="outer").groupby(["USER_ID", "Week_Number"]).count().reset_index()
weekly = weekly.merge(dimension, on=["USER_ID", "Week_Number"], how="right").fillna(0)
weekly = weekly.merge(first_visit_week, on ="USER_ID")
weekly = weekly.rename(columns={"ENTER_DATE": "VST_COUNT"})
weekly = weekly[weekly["first_visit_week"] <= weekly["Week_Number"]]
weekly = weekly[weekly["first_visit_week"] <= weekly["Week_Number"]]

summary_df = cstmr[["USER_ID"]]

for i in tqdm(range(1, 53)):
    temp = weekly[(weekly["Week_Number"] <= weekly["first_visit_week"]+i-1) & 
    (weekly["Week_Number"] >=weekly["first_visit_week"])].groupby("USER_ID")["VST_COUNT"].sum().reset_index()
    temp.rename(columns={"VST_COUNT": "VISITS_"+str(i)+"_WEEKS"}, inplace=True)
    summary_df = summary_df.merge(temp, on="USER_ID")

def calculate_survival_grace_days(group):
    # Ensure the group is sorted by Week_Number if not already
    group = group.sort_values('Week_Number')
    
    # Convert VST_COUNT to a string of 0s and 1s, where 0 indicates no visit
    visit_string = ''.join(group['VST_COUNT'].astype(int).astype(str))
    
    # Initialize the result dictionary
    results = {}
    
    weeks = 0
    if len(visit_string) > weeks:
        vst_string = visit_string[weeks:]
        grace_day_idx = vst_string.find('00')

        # If '0' is not found, use the length of the visit string
            
        if grace_day_idx == -1:
            grace_day_idx = len(vst_string)

    else:
        grace_day_idx = 0
        
    results[f'{weeks}_WEEKS_SURVIVAL_GRACE_DAY'] = grace_day_idx
    
    return pd.Series(results)

# Apply the function to each group and join the results back to the summary DataFrame
survival_grace_days = weekly.groupby('USER_ID').apply(calculate_survival_grace_days)
summary_df = summary_df.join(survival_grace_days, on='USER_ID', how="left")
summary_df.fillna(0, inplace=True)
cstmr = cstmr.merge(summary_df, on="USER_ID", how="left")


100%|██████████| 52/52 [00:50<00:00,  1.03it/s]


# Flags

In [25]:
flags = []
for i in range(6, 53):    
    left_side = cstmr[cstmr["0_WEEKS_SURVIVAL_GRACE_DAY"] <= i]
    right_side = cstmr[cstmr["0_WEEKS_SURVIVAL_GRACE_DAY"] > i]

    temp1 = []
    temp2 = []
    for visit in range(1, 7*i+1):
        temp1.append(len(left_side[left_side["VISITS_"+str(i)+"_WEEKS"] >= visit]) / len(left_side))
        temp2.append(len(right_side[right_side["VISITS_"+str(i)+"_WEEKS"] >= visit]) / len(right_side))

    difference = np.array(temp2) - np.array(temp1)
    max_diff = round(max(difference), 2)
    max_diff_index = np.argmax(difference)
    flags.append(max_diff_index)

for i, vst in enumerate(flags):
    cstmr["Flag_"+str(i+6)] = ((cstmr["VISITS_"+str(i+6)+"_WEEKS"] >= vst) & (cstmr["CONTRACT_LEN_WEEKS"] >= i+6)).astype(int)

# Prep for DoWhy

In [31]:
for i in tqdm(range(6, 18)):
    non_zero_values = cstmr[cstmr['GROUP_LESSONS_'+str(i)+'_WEEKS'] > 0]['GROUP_LESSONS_'+str(i)+'_WEEKS']
    quantiles = non_zero_values.quantile([1/3, 2/3])
    cstmr['GROUP_LESSONS_'+str(i)+'_WEEKS_0'] = (cstmr['GROUP_LESSONS_'+str(i)+'_WEEKS'] == 0).astype(int)
    cstmr['GROUP_LESSONS_'+str(i)+'_WEEKS_1'] = ((cstmr['GROUP_LESSONS_'+str(i)+'_WEEKS'] > 0)).astype(int)
    cstmr['GROUP_LESSONS_'+str(i)+'_WEEKS_2'] = ((cstmr['GROUP_LESSONS_'+str(i)+'_WEEKS'] > quantiles[1/3])).astype(int)
    cstmr['GROUP_LESSONS_'+str(i)+'_WEEKS_3'] = (cstmr['GROUP_LESSONS_'+str(i)+'_WEEKS'] > quantiles[2/3]).astype(int)
    
    
    non_zero_values = cstmr[cstmr["PT_"+str(i)+"_WEEKS"] > 0]["PT_"+str(i)+"_WEEKS"]
    quantiles = non_zero_values.quantile([1/3, 2/3])
    cstmr["PT_"+str(i)+"_WEEKS_0"] = (cstmr["PT_"+str(i)+"_WEEKS"] == 0).astype(int)
    cstmr["PT_"+str(i)+"_WEEKS_1"] = (cstmr["PT_"+str(i)+"_WEEKS"] > 0).astype(int)
    cstmr["PT_"+str(i)+"_WEEKS_2"] = (cstmr["PT_"+str(i)+"_WEEKS"] > quantiles[1/3]).astype(int)
    cstmr["PT_"+str(i)+"_WEEKS_3"] = (cstmr["PT_"+str(i)+"_WEEKS"] > quantiles[2/3]).astype(int)
    
    non_zero_values = cstmr[cstmr["DIFF_VST_"+str(i)] > 0]["DIFF_VST_"+str(i)]
    quantiles = non_zero_values.quantile([1/3, 2/3])
    cstmr["DIFF_VST_"+str(i)+'_0'] = (cstmr["DIFF_VST_"+str(i)] == 0).astype(int)
    cstmr["DIFF_VST_"+str(i)+'_1'] = (cstmr["DIFF_VST_"+str(i)] > 0).astype(int)
    cstmr["DIFF_VST_"+str(i)+'_2'] = (cstmr["DIFF_VST_"+str(i)] > quantiles[1/3]).astype(int)
    cstmr["DIFF_VST_"+str(i)+'_3'] = (cstmr["DIFF_VST_"+str(i)] > quantiles[2/3]).astype(int)
    
    non_zero_values = cstmr[cstmr["DIFF_GROUP_LESSON_"+str(i)] > 0]["DIFF_GROUP_LESSON_"+str(i)]
    quantiles = non_zero_values.quantile([1/3, 2/3])
    cstmr["DIFF_GROUP_LESSON_"+str(i)+'_0'] = (cstmr["DIFF_GROUP_LESSON_"+str(i)] == 0).astype(int)
    cstmr["DIFF_GROUP_LESSON_"+str(i)+'_1'] = (cstmr["DIFF_GROUP_LESSON_"+str(i)] > 0).astype(int)
    cstmr["DIFF_GROUP_LESSON_"+str(i)+'_2'] = (cstmr["DIFF_GROUP_LESSON_"+str(i)] > quantiles[1/3]).astype(int)
    cstmr["DIFF_GROUP_LESSON_"+str(i)+'_3'] = (cstmr["DIFF_GROUP_LESSON_"+str(i)] > quantiles[2/3]).astype(int)
    
    
    non_zero_values = cstmr[cstmr["CREDIT_"+str(i)+"_WEEKS"] > 0]["CREDIT_"+str(i)+"_WEEKS"]
    quantiles = non_zero_values.quantile([1/3, 2/3])
    cstmr['CREDIT_'+str(i)+'_WEEKS_0'] = (cstmr["CREDIT_"+str(i)+"_WEEKS"] == 0).astype(int)
    cstmr['CREDIT_'+str(i)+'_WEEKS_1'] = (cstmr["CREDIT_"+str(i)+"_WEEKS"] > 0).astype(int)
    cstmr['CREDIT_'+str(i)+'_WEEKS_2'] = (cstmr["CREDIT_"+str(i)+"_WEEKS"] > quantiles[1/3]).astype(int)
    cstmr['CREDIT_'+str(i)+'_WEEKS_3'] = (cstmr["CREDIT_"+str(i)+"_WEEKS"] > quantiles[2/3]).astype(int)

In [33]:
cols = ["TECRUBE", 'FORM', 'SIKLIK']

for i in cols:
    value_counts = cstmr[i].value_counts()
    cstmr.loc[cstmr[i].isin(value_counts[value_counts < 250].index), i] = np.nan
    cstmr[i] = cstmr[i].fillna("N/A")

temp_1 = pd.get_dummies(cstmr[["FORM", "TECRUBE", "SIKLIK"]], prefix=["FORM", "TECRUBE", "SIKLIK"], dtype=int)

temp_1["FORM>10"] = temp_1["FORM_30+"] | temp_1["FORM_10-30"]
temp_1["FORM>30"] = temp_1["FORM_30+"]
temp_1["TECRUBE>0"] = temp_1["TECRUBE_1-2"] | temp_1["TECRUBE_2+"] | temp_1["TECRUBE_<1"]
temp_1["TECRUBE>1"] = temp_1["TECRUBE_1-2"] | temp_1["TECRUBE_2+"]
temp_1["TECRUBE>2"] = temp_1["TECRUBE_2+"]
temp_1["SIKLIK>1"] = temp_1["SIKLIK_1-2"] | temp_1["SIKLIK_2+"]
temp_1["SIKLIK>2"] = temp_1["SIKLIK_2+"]

temp_1 = temp_1[["FORM>10", "FORM>30", "TECRUBE>0", "TECRUBE>1", "TECRUBE>2", "SIKLIK>1", "SIKLIK>2"]]

# Concatenate the dummy columns with the remaining columns
cstmr = pd.concat([cstmr, temp_1], axis=1)

In [42]:
cstmr.to_csv("cstmr_june_22_dowhy.csv", index=False)